In [ ]:
!pip install transformers==3.5.1
!pip install datasets
!pip install xmltodict 
!pip install torch==1.6.0
!pip install sklearn
!pip install tqdm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 5.1 MB/s 
     |████████████████████████████████| 880 kB 71.5 MB/s 
     |████████████████████████████████| 2.9 MB 65.2 MB/s 
     |████████████████████████████████| 1.1 MB 71.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=1e0ae946223e0560503d5a44e924b5024779c0289fe1d2391845e4081f9a79bc
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During han

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import datasets
from datasets import load_dataset

ds = load_dataset("text",data_files=["/content/drive/MyDrive/Tweets.csv"])


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-4a5a29de2d3d0388/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import xmltodict
import datasets
from datasets import Dataset, concatenate_datasets
import glob
import pandas as pd
import torch

In [ ]:
#!/usr/bin/env python
import xmltodict
import datasets
from datasets import Dataset, concatenate_datasets
import glob
import pandas as pd
import re



dataframe_list = []
#for file in glob.glob("/content/drive/MyDrive/all_earnings_sub/*.xml"):
for file in glob.glob("/content/drive/MyDrive/original_pubmed/*.xml"):
  #print(file)
  with open(file, 'r', encoding='utf-8') as file: #'pubmed_1_copy.xml'
      my_xml = file.read()
      clean_xml = re.sub("[&;]","",my_xml)
      xml_dict = xmltodict.parse(my_xml)
      df1 = pd.DataFrame(xml_dict, columns=xml_dict.keys())
      order_dict = df1["rst"].iloc[0]
      segments_list_of_dicts = order_dict["segment"]
      xml_dataframe = pd.DataFrame.from_dict(segments_list_of_dicts, orient='columns')
      selected_df = xml_dataframe[~xml_dataframe.isnull().any(axis=1)] #removes null
      selected_df.sort_values(by='@id',ascending=True)
      df_formatted = selected_df.assign(parent_text=selected_df["#text"].shift(1))
      df_formatted["parent_text"].fillna(value=df_formatted.iloc[0]["#text"],inplace=True) #fillna parent
      dataframe_list.append(df_formatted)
      #df_formatted["parent_text"].fillna(value="Introduction",inplace=True) #fillna parent
      dataframe_list.append(df_formatted)
      all_earnings_sub_df = pd.concat(dataframe_list)

#print(all_earnings_df.head(15))
print(len(all_earnings_sub_df))

In [ ]:
import pandas as pd
#l = ['michael','michael','alice','carter']
#pd.Series(l).astype('category').cat.codes.values

rel_labels = list(all_earnings_sub_df["@relname"])
rel_2_int = pd.Series(rel_labels).astype('category').cat.codes.values
print(rel_2_int)
#print(len(set(rel_2_int)))

0         neutral
1        positive
2         neutral
3        negative
4        negative
           ...   
14635    positive
14636    negative
14637     neutral
14638    negative
14639     neutral
Name: airline_sentiment, Length: 14640, dtype: object

In [ ]:
#hr_df['candidates'] = candidates

all_earnings_sub_df["int_relation"] = rel_2_int.astype(float)
all_earnings_sub_df["int_relation"] = torch.Tensor.long(torch.Tensor(rel_2_int))
all_earnings_sub_df.head(5)

In [ ]:
rel_labels = list(tweets["airline_sentiment"])
rel_2_int = pd.Series(rel_labels).astype('category').cat.codes.values
print(rel_2_int)
tweets["int_relation"] = rel_2_int.astype(float)
#tweets["airline_sentiment"].apply(lambda s: mymap.get(s) if s in mymap else s)

[1 2 1 ... 1 0 1]


In [ ]:
tweets

In [ ]:
from transformers import AutoTokenizer

scibert = "allenai/scibert_scivocab_uncased"
#roberta_model = "roberta-base"
#scibert_tokenizer = AutoModelForSequenceClassification.from_pretrained(scibert,num_labels=len(set(all_earnings_sub_df["int_relation"])))
scibert_tokenizer = AutoTokenizer.from_pretrained(scibert,add_prefix_space=True, use_fast=False)

In [ ]:
import datasets 
from datasets import Dataset

tweets_dataset = Dataset.from_pandas(tweets)
print(tweets_dataset)

Dataset({
    features: ['tweet_id', 'airline_sentiment', 'text', 'int_relation'],
    num_rows: 14640
})


In [ ]:
from transformers import DataCollatorWithPadding

def tokenize_function(example):
    return scibert_tokenizer(example["text"],truncation=True)

#this is the sentence pair tokenization ^^ 
tokenized_datasets = tweets_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=scibert_tokenizer)

#print(tokenized_datasets["input_ids"])

#input_ids = bio_tokenizer(sentence,truncation=True, add_special_tokens=True,max_length=512)['input_ids']

wordpieces = [scibert_tokenizer.decode(input_id) for input_id in tokenized_datasets["input_ids"]]
print(wordpieces)

ModuleNotFoundError: ignored

In [ ]:
tokenized_datasets.column_names
tokenized_datasets.set_format("torch")

NameError: ignored

In [ ]:
tokenized_datasets
tokenized_datasets = tokenized_datasets.remove_columns(["tweet_id","text","airline_sentiment"])
tokenized_datasets = tokenized_datasets.rename_column("int_relation","labels") #int_relation", "labels"#
#tokenized_datasets = tokenized_datasets.rename_column("airline_sentiment", "labels")

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 10980
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3660
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.train_test_split()

In [ ]:
#tokenized_datasets = tokenized_datasets.remove_columns(["tweet_id","text"])
#tokenized_datasets = tokenized_datasets.rename_column("airline_sentiment","labels") #int_relation", "labels"
#tokenized_datasets = tokenized_datasets.rename_column("airline_sentiment", "labels")

In [ ]:
tokenized_datasets

tokenized_datasets["train"]["labels"]

tensor([0., 0., 0.,  ..., 0., 2., 2.])

In [ ]:

from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)

for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([16]),
 'input_ids': torch.Size([16, 46]),
 'token_type_ids': torch.Size([16, 46]),
 'attention_mask': torch.Size([16, 46])}

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=16, collate_fn=data_collator
)

In [ ]:
from transformers import AutoConfig
config = AutoConfig #.from_pretrained("bert-base-cased")
config.classifier_dropout = 0.3

config._num_labels=22
config.num_hidden_layers=24
config._name_or_path = "allenai/scibert-base-uncased"
config.vocab_size = 30000

In [ ]:
import torch 
import torch.nn as nn

from transformers import AutoModel

#model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")


#criterion = torch.nn.CrossEntropyLoss().to(device)

class ScienceClassifier(nn.Module):
  def __init__(self):
    super(ScienceClassifier, self).__init__()
    self.AutoModel = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")
    self.drop0 = nn.Dropout(0.25)
    self.linear1 = nn.Linear(3072, 512)
    self.relu1 = nn.ReLU()
    self.drop1 = nn.Dropout(0.25)
    self.linear2 = nn.Linear(512, 3)
    self.relu2 = nn.Softmax()

  def forward(self, input_ids, attention_mask):
    outputs = self.AutoModel(input_ids, attention_mask)
    last_hidden_state = outputs[0]
    pooled_output = torch.cat(tuple([last_hidden_state[:, i] for i in [-4, -3, -2, -1]]), dim=-1)
    x = self.drop0(pooled_output)
    x = self.relu1(self.linear1(x))
    x = self.drop1(x)
    x = self.relu2(self.linear2(x))
    return x

In [ ]:
model = ScienceClassifier()

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"

In [ ]:
from tqdm.auto import tqdm 
from torch.optim import AdamW


optim = AdamW(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss(ignore_index=2)

def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100


In [ ]:
import torchmetrics 
from torchmetrics.functional import stat_scores
from torchmetrics import Accuracy
#target = torch.tensor([0, 1, 2, 3])
#preds = torch.tensor([0, 2, 1, 3])
#stat_scores(preds, target, reduce='macro', num_classes=2)


num_training_steps = 1
progress_bar = tqdm(range(num_training_steps))

running_acc = 0.
model.train()
for epoch in range(1):
  correct = 0
  for batch in train_dataloader:
      optim.zero_grad()
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      active_labels = labels.view(-1, labels)
      #print(labels)
      #counter += 1

      #labels = torch.sub(labels,1)
      #print(labels)
      output = model(input_ids, attention_mask)
      #print(output.size(), labels.size())
      loss = criterion(output, active_labels) # ERROR

      print(loss)
      loss.backward()
      optim.step()
      acc_t = compute_accuracy(output,labels)

      running_acc += (acc_t - running_acc) / (epoch + 1)
      print(running_acc)
      #print(labels)
      #correct += (output == labels)
      #print(correct / len(train_dataloader))
  #accuracy = 100 * correct / len(train_dataloader)
  #print("Accuracy = {}".format(accuracy))
      progress_bar.update(1)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.7181, grad_fn=<NllLossBackward>)
62.5
tensor(0.6348, grad_fn=<NllLossBackward>)
68.75
tensor(0.5514, grad_fn=<NllLossBackward>)
81.25
tensor(0.7822, grad_fn=<NllLossBackward>)
62.5
tensor(0.8014, grad_fn=<NllLossBackward>)
56.25
tensor(0.9361, grad_fn=<NllLossBackward>)
50.0
tensor(0.8848, grad_fn=<NllLossBackward>)
50.0
tensor(0.8848, grad_fn=<NllLossBackward>)
50.0
tensor(0.8181, grad_fn=<NllLossBackward>)
68.75
tensor(0.8014, grad_fn=<NllLossBackward>)
75.0
tensor(0.7514, grad_fn=<NllLossBackward>)
50.0
tensor(0.9361, grad_fn=<NllLossBackward>)
50.0
tensor(0.7822, grad_fn=<NllLossBackward>)
62.5
tensor(0.6181, grad_fn=<NllLossBackward>)
87.5
tensor(0.9514, grad_fn=<NllLossBackward>)
56.25
tensor(0.7657, grad_fn=<NllLossBackward>)
68.75
tensor(0.6943, grad_fn=<NllLossBackward>)
75.0
tensor(0.7822, grad_fn=<NllLossBackward>)
62.5
tensor(0.6229, grad_fn=<NllLossBackward>)
81.25
tensor(0.7657, grad_fn=<NllLossBackward>)
68.75
tensor(0.8014, grad_fn=<NllLossBackward>)
56.25
tens

In [ ]:
#target = torch.tensor([0, 1, 2, 3])
#preds = torch.tensor([0, 2, 1, 3])
#stat_scores(preds, target, reduce='macro', num_classes=2)


num_training_steps = 1
progress_bar = tqdm(range(num_training_steps))

model.eval()
running_acc = 0.
for epoch in range(1):
  correct = 0
  for batch in eval_dataloader:
      optim.zero_grad()
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      #print(labels)
      #counter += 1
    
      #labels = torch.sub(labels,1)
      labels = labels.to(torch.int64)
      #print(labels)
      output = model(input_ids, attention_mask)
      #print(output.size(), labels.size())
      loss = criterion(output, labels) # ERROR

      print(loss)
      loss.backward()
      optim.step()
      acc_t = compute_accuracy(output,labels)

      running_acc += (acc_t - running_acc) / (epoch + 1)
      print(running_acc)
      #print(labels)
      #correct += (output == labels)
      #print(correct / len(train_dataloader))
  #accuracy = 100 * correct / len(train_dataloader)
  #print("Accuracy = {}".format(accuracy))
      progress_bar.update(1)

  0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor(0.8181, grad_fn=<NllLossBackward>)
68.75
tensor(0.8372, grad_fn=<NllLossBackward>)
62.5
tensor(0.6943, grad_fn=<NllLossBackward>)
75.0
tensor(0.9681, grad_fn=<NllLossBackward>)
43.75
tensor(0.8181, grad_fn=<NllLossBackward>)
68.75
tensor(0.8639, grad_fn=<NllLossBackward>)
68.75
tensor(0.6943, grad_fn=<NllLossBackward>)
75.0
tensor(0.7822, grad_fn=<NllLossBackward>)
62.5
tensor(0.7822, grad_fn=<NllLossBackward>)
62.5
tensor(0.5514, grad_fn=<NllLossBackward>)
75.0
tensor(0.6229, grad_fn=<NllLossBackward>)
81.25
tensor(0.7657, grad_fn=<NllLossBackward>)
68.75
tensor(0.6181, grad_fn=<NllLossBackward>)
87.5
tensor(0.6848, grad_fn=<NllLossBackward>)
81.25
tensor(0.7053, grad_fn=<NllLossBackward>)
68.75
tensor(0.7657, grad_fn=<NllLossBackward>)
68.75
tensor(0.6848, grad_fn=<NllLossBackward>)
81.25
tensor(0.7514, grad_fn=<NllLossBackward>)
75.0
tensor(0.6848, grad_fn=<NllLossBackward>)
81.25
tensor(0.5514, grad_fn=<NllLossBackward>)
68.75
tensor(0.7657, grad_fn=<NllLossBackward>)
68.75
